# DEFINING AGENT
## Software entities that can autononously perform tasks

common characteristics of an Agent-
- Autonomous
- Goal-Oriented
- Task specific

Designed to work as part of an Agent Framework to solve complex problems with limited human involvement-
- Memory/persistence
- Decision-making/orchestration
- plannig capabilities
- Use of tools; potentially connecting to database ot the internet

## What we are about to do
- Image Generation
  - Use the OpenAI interface to generate images
- Make Agents
  - Crate Agents to generate sound and images for our store.
- Make an Agent Framework
  - Teach our AI Assistant to speak and draw.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
MODEL = "llama3.2:1b"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [2]:
response= openai.chat.completions.create(
    model=MODEL,
    messages=[{"role":"system", "content":"You are a helpful Assistant."},
             {"role":"user", "content":"What does paradise mean."}]
)
response.choices[0].message.content

"Paradise is a place of perfect peace, happiness, and beauty, often associated with a heavenly or divine location. In many cultures and religious traditions, paradise refers to a state of being that is characterized by extraordinary joy, serenity, and fulfillment.\n\nIn a general sense, paradise can refer to any place where one feels completely comfortable, content, and at ease. It's like a warm embrace that wraps around the soul, making everything feel just right. Whether it's a physical location on earth or an idealized concept in the imagination, paradise implies a state of bliss that is often associated with a sense of eternity.\n\nIn religious contexts, paradise typically refers to a future state of existence where individuals will be reunited with their loved ones who have passed away, and where they will experience perfect happiness, joy, and divine love. Some example references to paradise include:\n\n* The Garden of Eden in the Bible, often seen as a representation of an ideal

## Let's go multi-modal!!
We can use DALL-E-3, the image genaration model behind GPT-40, to make us some images.
Let's put this in a function called artist.

### Price alert: each time i generate an image it costs about 4c- don't go crazy with images!

In [7]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [8]:
# initialization

load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
MODEL="gpt-4o-mini"
openai=OpenAI()

In [9]:
# some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [24]:
def artist(city):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64=image_response.data[0].b64_json
    image_data=base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [26]:
image=artist("New York City")
display(image)

## AUDIO
And let's make a function talker that uses OpenAI's speech model to generate Audio

In [27]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # alloy & onyx
        input=message
    )

    audio_stream=BytesIO(response.content)
    audio=AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

C:\Users\hp\.conda\envs\llms1\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [29]:
talker("Well, hi there")

## Our Agent Framework
The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

- Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
- The ability for LLMs to use Tools to give them additional capabilities
- The 'Agent Environment' which allows Agents to collaborate
- An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
- The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

We're showing 1 and 2 here, and to a lesser extent 3 and 5. In week 8 we will do the lot!

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

## Exercises and Business Applications
- Add in more tools - perhaps to simulate actually booking a flight. A student has done this and provided their example in the community contributions folder.

- Next: take this and apply it to your business. Make a multi-modal AI assistant with tools that could carry out an activity for your work. A customer support assistant? New employee onboarding assistant? So many possibilities! Also, see the week2 end of week Exercise in the separate Notebook.